In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/zip-ragbot/ragbot_database/ragbot_database/lora_adapter_latest/adapter_model.safetensors
/kaggle/input/zip-ragbot/ragbot_database/ragbot_database/lora_adapter_latest/adapter_config.json
/kaggle/input/zip-ragbot/ragbot_database/ragbot_database/lora_adapter_latest/README.md
/kaggle/input/zip-ragbot/ragbot_database/ragbot_database/lora_adapter_latest/tokenizer.json
/kaggle/input/zip-ragbot/ragbot_database/ragbot_database/lora_adapter_latest/tokenizer_config.json
/kaggle/input/zip-ragbot/ragbot_database/ragbot_database/lora_adapter_latest/chat_template.jinja
/kaggle/input/zip-ragbot/ragbot_database/ragbot_database/lora_adapter_latest/special_tokens_map.json
/kaggle/input/zip-ragbot/ragbot_database/ragbot_database/qdrant_db/meta.json
/kaggle/input/zip-ragbot/ragbot_database/ragbot_database/qdrant_db/collection/your_collection/storage.sqlite


In [2]:
!pip install qdrant-client
!pip install peft transformers accelerate bitsandbytes --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 23.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━

In [3]:
! pip install -U bitsandbytes

In [4]:
import bitsandbytes as bnb
print(bnb.__version__)


0.46.1


In [5]:
!pip install -U peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.3/472.3 kB 12.5 MB/s eta 0:00:0000:01
  Attempting uninstall: peft
    Found existing installation: peft 0.15.2
    Uninstalling peft-0.15.2:
      Successfully uninstalled peft-0.15.2


In [6]:
import os
import torch
# from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import torch
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, Distance, VectorParams
import uuid
import shutil

2025-07-31 05:13:14.423593: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753938794.601099      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753938794.657562      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("HF_TOKEN")
from huggingface_hub import login
login(token=token)

In [8]:
# import os

# lock_path = "/kaggle/input/ragbot-database/ragbot_database/qdrant_db/.lock"
# try:
#     os.remove(lock_path)
#     print("Lock file removed.")
# except FileNotFoundError:
#     print("No lock file found.")


In [9]:
! ls "/kaggle/input/d/meghaguptastufit/ragbot-database/ragbot_database/lora_adapter_latest"

ls: cannot access '/kaggle/input/d/meghaguptastufit/ragbot-database/ragbot_database/lora_adapter_latest': No such file or directory


In [10]:
import zipfile
import shutil

# Copy zip to working (if needed)
shutil.copytree("/kaggle/input/zip-ragbot/ragbot_database/ragbot_database/", "/kaggle/working/ragbot_database/")

# # Unzip in working directory
# with zipfile.ZipFile("/kaggle/working/archive.zip", 'r') as zip_ref:
#     zip_ref.extractall("/kaggle/working/your_folder")


'/kaggle/working/ragbot_database/'

In [11]:
# # Qdrant connection
# qdrant = QdrantClient(path="vector_store_api/qdrant_db")

# collection_name = "my_collection"

# # Ensure collection exists (matching your folder)
# if collection_name not in [c.name for c in qdrant.get_collections().collections]:
#     qdrant.create_collection(
#         collection_name=collection_name,
#         vectors_config=models.VectorParams(
#             size=embedding_dim,
#             distance=models.Distance.COSINE
#         )
#     )

# Initialize Qdrant
qdrant = QdrantClient(path="/kaggle/working/ragbot_database/qdrant_db")  # Or path/to/db if persistent
collection_name = "your_collection"

# Connect to existing Qdrant vector database
# qdrant = QdrantClient(path="/content/drive/MyDrive/vector_store_api/qdrant_db")
# collection_name = "my_collection"  # collection name from pipeline

# Check if collection exists
collections = [c.name for c in qdrant.get_collections().collections]
if collection_name not in collections:
    raise ValueError(f"Collection '{collection_name}' not found. Please run the ingestion pipeline first.")

# Load the embedding model (same one used during indexing)
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # or the one you used




modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
adapter_path = "/kaggle/working/ragbot_database/lora_adapter_latest"

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [13]:
# Load model & tokenizer
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id, token=token, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto", token=token)

# Load and merge the adapter
model = PeftModel.from_pretrained(model, adapter_path)
# model = model.merge_and_unload()  # Merge adapter weights into base model
model.eval()

# Create text generation pipeline
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)




tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [14]:
# Prompt builder
def build_prompt(user_query, retrieved_chunks):
    joined_chunks = "\n\n".join(retrieved_chunks)
    instruction = f"""
You are a helpful medical assistant. Analyze the following medical report and retrieved expert medical knowledge. Identify any possible health issues or important insights.

Medical Report:
{user_query}

Relevant Medical Knowledge:
{joined_chunks}
\n\n
Answer (Only use the above relevant knowledge. If not found, reply "I don't know."):
"""
    return f"<s>[INST] {instruction.strip()} [/INST]"




In [15]:
# vector search + generate response

def generate_response(user_query, top_k=5):
    # Step 1: Embed user query
    query_embedding = embedding_model.encode([user_query])[0].tolist()

    # Step 2: Vector search in Qdrant
    search_result = qdrant.query_points(
        collection_name=collection_name,
        query=query_embedding,
        limit=top_k,
        with_payload=True
    )
    
    # Step 3: Collect retrieved documents
    retrieved_chunks = []
    for hit in search_result:
        # Some APIs wrap ScoredPoint inside a list or return a list of lists
        if isinstance(hit, list):
            for item in hit:
                if hasattr(item, "payload"):
                    payload = item.payload
                    if payload:
                        retrieved_chunks.append(
                            payload.get("text") or payload.get("chunk") or payload.get("content")
                        )
        elif hasattr(hit, "payload"):
            payload = hit.payload
            if payload:
                retrieved_chunks.append(
                    payload.get("text") or payload.get("chunk") or payload.get("content")
                )

    if not retrieved_chunks:
        return "I don't know."

    # Step 4: Create prompt from context + query
    context = "\n".join(retrieved_chunks)
    prompt = f"""You are an AI assistant. Use the following context to answer the question.

Context:
{context}

Question: {user_query}
Answer:"""

    # Step 5: Generate response using merged model
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=300, do_sample=True, temperature=0.7)
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Step 6: Save query+response to vector DB
    combined_text = f"Prompt: {user_query}\nAnswer: {response_text}"
    combined_embedding = embedding_model.encode([combined_text])[0].tolist()

    qdrant.upsert(
        collection_name=collection_name,
        points=[
            PointStruct(id=str(uuid.uuid4()), vector=combined_embedding, payload={"text": combined_text})
        ]
    )

    # Step 7: Return response to user
    return response_text


In [16]:
# View existing documents in Qdrant
search = qdrant.scroll(
    collection_name=collection_name,
    limit=5,
    with_payload=True
)

for point in search[0]:
    print("ID:", point.id)
    print("Payload:", point.payload)
    print("---")


ID: 0
Payload: {}
---


In [17]:
print(qdrant.count(collection_name=collection_name).count)


1


In [18]:
test_text = """
A 62-year-old patient with persistent headaches, high blood pressure (165/105), 
elevated serum creatinine (1.6 mg/dL), and protein in the urine may be showing signs of 
hypertensive nephropathy or early kidney damage.
"""

test_embedding = embedding_model.encode([test_text])[0].tolist()

qdrant.upsert(
    collection_name=collection_name,
    points=[
        PointStruct(id=str(uuid.uuid4()), vector=test_embedding, payload={"text": test_text})
    ]
)

print("Test document inserted.")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Test document inserted.


In [19]:
query = """
Patient Report:
- Age: 62
- Symptoms: Persistent headaches, dizziness, occasional vision issues
- Blood Pressure: 165/105 mmHg
- Serum Creatinine: 1.6 mg/dL
- Urinalysis: Presence of protein
- History: Long-standing untreated high blood pressure

Please analyze and highlight any significant findings or possible diagnoses.
"""
answer = generate_response(query)
print("Generated Answer:\n", answer)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated Answer:
 I don't know.
